# 分组和数据透视表

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.DataFrame({"key":["A", "B", "C", "C", "B", "A"],
                  "data1": range(6),
                  "data2": np.random.randint(0, 10, size=6)})
df

,key,data1,data2
0,A,0,6
1,B,1,6
2,C,2,4
3,C,3,0
4,B,4,1
5,A,5,3


**（1）分组**

* 延迟计算

In [5]:
df.groupby("key")

In [6]:
df.groupby("key").sum()

,data1,data2
key,,
A,5,9
B,5,7
C,5,4


In [7]:
df.groupby("key").mean()

,data1,data2
key,,
A,2.5,4.5
B,2.5,3.5
C,2.5,2.0


In [8]:
for i in df.groupby("key"):
    print(str(i))

('A',   key  data1  data2
0   A      0      6
5   A      5      3)
('B',   key  data1  data2
1   B      1      6
4   B      4      1)
('C',   key  data1  data2
2   C      2      4
3   C      3      0)


* 按列取值

In [20]:
df.groupby("key")["data2"].sum()

key
A    10
B     6
C    11
Name: data2, dtype: int32

* 按组迭代

In [21]:
for data, group in df.groupby("key"):
    print("{0:5} shape={1}".format(data, group.shape))

A     shape=(2, 3)
B     shape=(2, 3)
C     shape=(2, 3)


* 调用方法

In [22]:
df.groupby("key")["data1"].describe()

,count,mean,std,min,25%,50%,75%,max
key,,,,,,,,
A,2.0,2.5,3.535534,0.0,1.25,2.5,3.75,5.0
B,2.0,2.5,2.121320,1.0,1.75,2.5,3.25,4.0
C,2.0,2.5,0.707107,2.0,2.25,2.5,2.75,3.0


* 支持更复杂的操作

In [23]:
df.groupby("key").aggregate(["min", "median", "max"])

data1            data2           
      min median max   min median max
key                                  
A       0    2.5   5     2    5.0   8
B       1    2.5   4     2    3.0   4
C       2    2.5   3     3    5.5   8

* 过滤

In [24]:
def filter_func(x):
    return x["data2"].std() > 3
df.groupby("key")["data2"].std()

key
A    4.242641
B    1.414214
C    3.535534
Name: data2, dtype: float64

In [25]:
df.groupby("key").filter(filter_func)

,key,data1,data2
0,A,0,2
2,C,2,8
3,C,3,3
5,A,5,8


* 转换

In [26]:
df

,key,data1,data2
0,A,0,2
1,B,1,2
2,C,2,8
3,C,3,3
4,B,4,4
5,A,5,8


In [28]:
df.groupby("key").transform(lambda x: x-x.mean())

,data1,data2
0,-2.5,-3.0
1,-1.5,-1.0
2,-0.5,2.5
3,0.5,-2.5
4,1.5,1.0
5,2.5,3.0


In [104]:
df

,key,data1,data2
0,A,0,1
1,B,1,4
2,C,2,9
3,C,3,9
4,B,4,1
5,A,5,9


In [105]:
df.groupby("key").apply(lambda x: x-x.mean())

,data1,data2
0,-2.5,-4.0
1,-1.5,1.5
2,-0.5,0.0
3,0.5,0.0
4,1.5,-1.5
5,2.5,4.0


* apply（）方法

In [31]:
df

,key,data1,data2
0,A,0,2
1,B,1,2
2,C,2,8
3,C,3,3
4,B,4,4
5,A,5,8


In [29]:
def norm_by_data2(x):
    x["data1"] /= x["data2"].sum()
    return x

In [30]:
df.groupby("key").apply(norm_by_data2)

,key,data1,data2
0,A,0.000000,2
1,B,0.166667,2
2,C,0.181818,8
3,C,0.272727,3
4,B,0.666667,4
5,A,0.500000,8


* 将列表、数组设为分组键

In [32]:
L = [0, 1, 0, 1, 2, 0]
df

,key,data1,data2
0,A,0,2
1,B,1,2
2,C,2,8
3,C,3,3
4,B,4,4
5,A,5,8


In [33]:
df.groupby(L).sum()

,data1,data2
0,7,18
1,4,5
2,4,4


* 用字典将索引映射到分组

In [34]:
df2 = df.set_index("key")
df2

,data1,data2
key,,
A,0,2
B,1,2
C,2,8
C,3,3
B,4,4
A,5,8


In [35]:
mapping = {"A": "first", "B": "constant", "C": "constant"}
df2.groupby(mapping).sum()

,data1,data2
constant,10,17
first,5,10


* 任意Python函数

In [36]:
df2.groupby(str.lower).mean()

,data1,data2
a,2.5,5.0
b,2.5,3.0
c,2.5,5.5


* 多个有效值组成的列表

In [37]:
df2.groupby([str.lower, mapping]).mean()

,,data1,data2
a,first,2.5,5.0
b,constant,2.5,3.0
c,constant,2.5,5.5


【例1】 行星观测数据处理

In [9]:
import seaborn as sns

planets = sns.load_dataset("planets")

URLError: <urlopen error [Errno 11004] getaddrinfo failed>

In [40]:
planets.shape

(1035, 6)

In [47]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [48]:
planets.describe()

,number,orbital_period,mass,distance,year
count,1035.000000,992.000000,513.000000,808.000000,1035.000000
mean,1.785507,2002.917596,2.638161,264.069282,2009.070531
std,1.240976,26014.728304,3.818617,733.116493,3.972567
min,1.000000,0.090706,0.003600,1.350000,1989.000000
25%,1.000000,5.442540,0.229000,32.560000,2007.000000
50%,1.000000,39.979500,1.260000,55.250000,2010.000000
75%,2.000000,526.005000,3.040000,178.500000,2012.000000
max,7.000000,730000.000000,25.000000,8500.000000,2014.000000


In [49]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [51]:
decade = 10 * (planets["year"] // 10)
decade.head()

0    2000
1    2000
2    2010
3    2000
4    2000
Name: year, dtype: int64

In [52]:
decade = decade.astype(str) + "s"
decade.name = "decade"
decade.head()

0    2000s
1    2000s
2    2010s
3    2000s
4    2000s
Name: decade, dtype: object

In [55]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [53]:
planets.groupby(["method", decade]).sum()

number  orbital_period       mass  \
method                        decade                                      
Astrometry                    2010s        2    1.262360e+03    0.00000   
Eclipse Timing Variations     2000s        5    1.930800e+04    6.05000   
                              2010s       10    2.345680e+04    4.20000   
Imaging                       2000s       29    1.350935e+06    0.00000   
                              2010s       21    6.803750e+04    0.00000   
Microlensing                  2000s       12    1.732500e+04    0.00000   
                              2010s       15    4.750000e+03    0.00000   
Orbital Brightness Modulation 2010s        5    2.127920e+00    0.00000   
Pulsar Timing                 1990s        9    1.900153e+02    0.00000   
                              2000s        1    3.652500e+04    0.00000   
                              2010s        1    9.070629e-02    0.00000   
Pulsation Timing Variations   2000s        1    1.170000e+03    0.00000   
Radial Velocity               1980s        1    8.388800e+01   11.68000   
                              1990s       52    1.091561e+04   68.17820   
                              2000s      475    2.633526e+05  945.31928   
                              2010s      424    1.809630e+05  316.47890   
Transit                       2000s       64    2.897102e+02    0.00000   
                              2010s      712    8.087813e+03    1.47000   
Transit Timing Variations     2010s        9    2.393505e+02    0.00000   

                                       distance    year  
method                        decade                     
Astrometry                    2010s       35.75    4023  
Eclipse Timing Variations     2000s      261.44    6025  
                              2010s     1000.00   12065  
Imaging                       2000s      956.83   40139  
                              2010s     1210.08   36208  
Microlensing                  2000s        0.00   20070  
                              2010s    41440.00   26155  
Orbital Brightness Modulation 2010s     2360.00    6035  
Pulsar Timing                 1990s        0.00    5978  
                              2000s        0.00    2003  
                              2010s     1200.00    2011  
Pulsation Timing Variations   2000s        0.00    2007  
Radial Velocity               1980s       40.57    1989  
                              1990s      723.71   55943  
                              2000s    15201.16  619775  
                              2010s    11382.67  432451  
Transit                       2000s    31823.31  124462  
                              2010s   102419.46  673999  
Transit Timing Variations     2010s     3313.00    8050

In [56]:
planets.groupby(["method", decade])[["number"]].sum().unstack().fillna(0)

number                    
decade                         1980s 1990s  2000s  2010s
method                                                  
Astrometry                       0.0   0.0    0.0    2.0
Eclipse Timing Variations        0.0   0.0    5.0   10.0
Imaging                          0.0   0.0   29.0   21.0
Microlensing                     0.0   0.0   12.0   15.0
Orbital Brightness Modulation    0.0   0.0    0.0    5.0
Pulsar Timing                    0.0   9.0    1.0    1.0
Pulsation Timing Variations      0.0   0.0    1.0    0.0
Radial Velocity                  1.0  52.0  475.0  424.0
Transit                          0.0   0.0   64.0  712.0
Transit Timing Variations        0.0   0.0    0.0    9.0

**（2）数据透视表**

【例2】泰坦尼克号乘客数据分析

In [44]:
import seaborn as sns

titanic = sns.load_dataset("titanic")

In [45]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [51]:
T = titanic[titanic.age.notnull()].copy()

In [59]:
T.age.apply(lambda x: 60 if x>=60 else x)
T.age.value_counts()

24.00    30
22.00    27
60.00    26
18.00    26
28.00    25
30.00    25
19.00    25
21.00    24
25.00    23
36.00    22
29.00    20
35.00    18
32.00    18
27.00    18
26.00    18
31.00    17
16.00    17
34.00    15
20.00    15
33.00    15
23.00    15
39.00    14
40.00    13
17.00    13
42.00    13
45.00    12
38.00    11
4.00     10
50.00    10
2.00     10
         ..
8.00      4
5.00      4
11.00     4
6.00      3
7.00      3
46.00     3
30.50     2
57.00     2
0.83      2
55.00     2
10.00     2
59.00     2
13.00     2
28.50     2
40.50     2
45.50     2
0.75      2
32.50     2
34.50     1
55.50     1
0.92      1
36.50     1
12.00     1
53.00     1
14.50     1
0.67      1
20.50     1
23.50     1
24.50     1
0.42      1
Name: age, Length: 77, dtype: int64

In [60]:
Age = 10*(T["age"]//10)
Age = Age.astype(int)
Age.head()
Age.value_counts()

20    220
30    167
10    102
40     89
0      62
50     48
60     26
Name: age, dtype: int64

In [61]:
Age.astype(str)+"s"

0      20s
1      30s
2      20s
3      30s
4      30s
6      50s
7       0s
8      20s
9      10s
10      0s
11     50s
12     20s
13     30s
14     10s
15     50s
16      0s
18     30s
20     30s
21     30s
22     10s
23     20s
24      0s
25     30s
27     10s
30     40s
33     60s
34     20s
35     40s
37     20s
38     10s
      ... 
856    40s
857    50s
858    20s
860    40s
861    20s
862    40s
864    20s
865    40s
866    20s
867    30s
869     0s
870    20s
871    40s
872    30s
873    40s
874    20s
875    10s
876    20s
877    10s
879    50s
880    20s
881    30s
882    20s
883    20s
884    20s
885    30s
886    20s
887    10s
889    20s
890    30s
Name: age, Length: 714, dtype: object

In [63]:
T.groupby(["sex", Age])["survived"].mean().unstack()

age,0,10,20,30,40,50,60
sex,,,,,,,
female,0.633333,0.755556,0.722222,0.833333,0.687500,0.888889,1.000000
male,0.593750,0.122807,0.168919,0.214953,0.210526,0.133333,0.136364


In [66]:
T.age = Age
T.pivot_table("survived", index="sex", columns="age")

age,0,10,20,30,40,50,60
sex,,,,,,,
female,0.633333,0.755556,0.722222,0.833333,0.687500,0.888889,1.000000
male,0.593750,0.122807,0.168919,0.214953,0.210526,0.133333,0.136364


In [59]:
titanic.describe()

,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [60]:
titanic.groupby("sex")[["survived"]].mean()

,survived
sex,
female,0.742038
male,0.188908


In [61]:
titanic.groupby("sex")["survived"].mean()

sex
female    0.742038
male      0.188908
Name: survived, dtype: float64

In [62]:
titanic.groupby(["sex", "class"])["survived"].aggregate("mean").unstack()

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


* 数据透视表

In [63]:
titanic.pivot_table("survived", index="sex", columns="class")

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


In [64]:
titanic.pivot_table("survived", index="sex", columns="class", aggfunc="mean", margins=True)

class,First,Second,Third,All
sex,,,,
female,0.968085,0.921053,0.500000,0.742038
male,0.368852,0.157407,0.135447,0.188908
All,0.629630,0.472826,0.242363,0.383838


In [65]:
titanic.pivot_table(index="sex", columns="class", aggfunc={"survived": "sum", "fare": "mean"})

fare                       survived             
class        First     Second      Third    First Second Third
sex                                                           
female  106.125798  21.970121  16.118810       91     70    72
male     67.226127  19.741782  12.661633       45     17    47

## 12.7 其他

**（1）向量化字符串操作**

**（2） 处理时间序列**

**（3） 多级索引：用于多维数据**

In [66]:
base_data = np.array([[1771, 11115 ],
                      [2154, 30320],
                      [2141, 14070],
                      [2424, 32680],
                      [1077, 7806],
                      [1303, 24222],
                      [798, 4789],
                      [981, 13468]]) 
data = pd.DataFrame(base_data, index=[["BeiJing","BeiJing","ShangHai","ShangHai","ShenZhen","ShenZhen","HangZhou","HangZhou"]\
                                     , [2008, 2018]*4], columns=["population", "GDP"])
data

population    GDP
BeiJing  2008        1771  11115
         2018        2154  30320
ShangHai 2008        2141  14070
         2018        2424  32680
ShenZhen 2008        1077   7806
         2018        1303  24222
HangZhou 2008         798   4789
         2018         981  13468

In [67]:
data.index.names = ["city", "year"]
data

population    GDP
city     year                   
BeiJing  2008        1771  11115
         2018        2154  30320
ShangHai 2008        2141  14070
         2018        2424  32680
ShenZhen 2008        1077   7806
         2018        1303  24222
HangZhou 2008         798   4789
         2018         981  13468

In [70]:
data["GDP"]

city      year
BeiJing   2008    11115
          2018    30320
ShangHai  2008    14070
          2018    32680
ShenZhen  2008     7806
          2018    24222
HangZhou  2008     4789
          2018    13468
Name: GDP, dtype: int32

In [71]:
data.loc["ShangHai", "GDP"]

year
2008    14070
2018    32680
Name: GDP, dtype: int32

In [73]:
data.loc["ShangHai", 2018]["GDP"]

32680

**（4） 高性能的Pandas：eval（）**

In [75]:
df1, df2, df3, df4 = (pd.DataFrame(np.random.random((10000,100))) for i in range(4))

In [76]:
%timeit (df1+df2)/(df3+df4)

17.6 ms ± 120 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


* 减少了复合代数式计算中间过程的内存分配

In [77]:
%timeit pd.eval("(df1+df2)/(df3+df4)")

10.5 ms ± 153 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [78]:
np.allclose((df1+df2)/(df3+df4), pd.eval("(df1+df2)/(df3+df4)"))

True

* 实现列间运算

In [79]:
df = pd.DataFrame(np.random.random((1000, 3)), columns=list("ABC"))
df.head()

,A,B,C
0,0.418071,0.381836,0.500556
1,0.059432,0.749066,0.302429
2,0.489147,0.739153,0.777161
3,0.175441,0.016556,0.348979
4,0.766534,0.559252,0.310635


In [80]:
res_1 = pd.eval("(df.A+df.B)/(df.C-1)")

In [81]:
res_2 = df.eval("(A+B)/(C-1)")

In [82]:
np.allclose(res_1, res_2)

True

In [83]:
df["D"] = pd.eval("(df.A+df.B)/(df.C-1)")
df.head()

,A,B,C,D
0,0.418071,0.381836,0.500556,-1.601593
1,0.059432,0.749066,0.302429,-1.159019
2,0.489147,0.739153,0.777161,-5.512052
3,0.175441,0.016556,0.348979,-0.294917
4,0.766534,0.559252,0.310635,-1.923199


In [84]:
df.eval("D=(A+B)/(C-1)", inplace=True)
df.head()

,A,B,C,D
0,0.418071,0.381836,0.500556,-1.601593
1,0.059432,0.749066,0.302429,-1.159019
2,0.489147,0.739153,0.777161,-5.512052
3,0.175441,0.016556,0.348979,-0.294917
4,0.766534,0.559252,0.310635,-1.923199


* 使用局部变量

In [85]:
column_mean = df.mean(axis=1)
res = df.eval("A+@column_mean")
res.head()

0    0.342788
1    0.047409
2   -0.387501
3    0.236956
4    0.694839
dtype: float64

**（4） 高性能的Pandas：query（）**

In [86]:
df.head()

,A,B,C,D
0,0.418071,0.381836,0.500556,-1.601593
1,0.059432,0.749066,0.302429,-1.159019
2,0.489147,0.739153,0.777161,-5.512052
3,0.175441,0.016556,0.348979,-0.294917
4,0.766534,0.559252,0.310635,-1.923199


In [87]:
%timeit df[(df.A < 0.5) & (df.B > 0.5)]

1.11 ms ± 9.38 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [88]:
%timeit df.query("(A < 0.5)&(B > 0.5)")

2.55 ms ± 199 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [97]:
df.query("(A < 0.5)&(B > 0.5)").head()

,A,B,C,D
1,0.059432,0.749066,0.302429,-1.159019
2,0.489147,0.739153,0.777161,-5.512052
7,0.073950,0.730144,0.646190,-2.272672
10,0.393200,0.610467,0.697096,-3.313485
11,0.065734,0.764699,0.179380,-1.011958


In [98]:
np.allclose(df[(df.A < 0.5) & (df.B > 0.5)], df.query("(A < 0.5)&(B > 0.5)"))

True

**（5）eval（）和query（）的使用时机**

小数组时，普通方法反而更快

In [99]:
df.values.nbytes

32000

In [100]:
df1.values.nbytes

8000000